In [1]:
import pandas as pd
import numpy as np


In [155]:
total = pd.read_csv('total.csv')
count = total.groupby('event_subtype')['file'].count()
subtype = total['event_subtype'].unique()

In [158]:
subtype

array(['Transport', 'Elect', 'Start-Position', 'Nominate', 'Attack',
       'End-Position', 'Meet', 'Marry', 'Phone-Write', 'Transfer-Money',
       'Sue', 'Demonstrate', 'End-Org', 'Injure', 'Die', 'Arrest-Jail',
       'Transfer-Ownership', 'Start-Org', 'Execute', 'Trial-Hearing',
       'Sentence', 'Be-Born', 'Charge-Indict', 'Convict',
       'Declare-Bankruptcy', 'Release-Parole', 'Fine', 'Appeal',
       'Merge-Org', 'Extradite', 'Divorce', 'Acquit'], dtype=object)

In [157]:
subtype = np.delete(subtype,np.where(subtype == 'Pardon')[0][0])

In [159]:
np.random.shuffle(subtype)
total_classes = len(subtype)
base_classes = 7
inc_classes = 5
sessions = 1 + (total_classes - base_classes) // inc_classes

In [160]:
first_session_subtype = np.array(count[count.index.isin(count.sort_values().index[-7:])].index)
subtype = np.setdiff1d(subtype,first_session_subtype)
class_split = [(s - 1) * inc_classes + np.arange(inc_classes) for s in range(1, sessions)]
sessions_subtype = [subtype[cs] for cs in class_split]

In [161]:
sessions_subtype

[array(['Acquit', 'Appeal', 'Arrest-Jail', 'Be-Born', 'Charge-Indict'],
       dtype=object),
 array(['Convict', 'Declare-Bankruptcy', 'Demonstrate', 'Divorce',
        'End-Org'], dtype=object),
 array(['Execute', 'Extradite', 'Fine', 'Injure', 'Marry'], dtype=object),
 array(['Merge-Org', 'Nominate', 'Phone-Write', 'Release-Parole',
        'Sentence'], dtype=object),
 array(['Start-Org', 'Start-Position', 'Sue', 'Transfer-Ownership',
        'Trial-Hearing'], dtype=object)]

In [162]:
train = None
test = None
for current_subtype in first_session_subtype:
    session_rows = total[total.event_subtype == current_subtype]
    mask = np.random.rand(len(session_rows)) < 0.8
    train = session_rows[mask] if train is None else pd.concat([train,session_rows[mask]])
    test = session_rows[~mask] if test is None else pd.concat([test,session_rows[~mask]])

train['session'] = 0
test['session'] = 0
train['type'] = 'train'
test['type'] = 'test'
total_labelled = train
total_labelled = total_labelled.append(test)

In [163]:
session = 1
for session_subtype in sessions_subtype:
    for current_subtype in session_subtype:
        session_rows = total[total.event_subtype == current_subtype]
        train = session_rows.sample(5)
        test_index = np.setdiff1d(session_rows.index,train.index)
        test = session_rows.loc[test_index]

        train['session'] = session
        test['session'] = session
        train['type'] = 'train'
        test['type'] = 'test'
        total_labelled = total_labelled.append(train).append(test)
    session+=1

In [175]:
total_labelled.to_csv('total_labelled.csv',index=None)

In [2]:
total_labelled = pd.read_csv('total_labelled.csv')

In [4]:
train = total_labelled[total_labelled.type == 'train']
test = total_labelled[total_labelled.type == 'test']

In [5]:
train.to_csv('train.csv',index=None)
test.to_csv('test.csv',index=None)